In [12]:
# imports

from surfacecode.logical_qubit import LQubit
from surfacecode.circuits import SquareLattice
from surfacecode.lattice import *
from surfacecode.simulators import *
from surfacecode.surface import *
from surfacecode.mwpm import ParityCheckMatrix, ErrorCorrection

from qiskit_aer import AerSimulator
from qiskit import transpile
from qiskit_aer.noise import (NoiseModel, QuantumError, ReadoutError,
    pauli_error, depolarizing_error, thermal_relaxation_error)
from qiskit_ibm_provider import IBMProvider, least_busy

import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm


In [13]:
WIDTH, HEIGHT = 3, 9
l = (7, 19)

lattice = SquareLattice(WIDTH, HEIGHT)
qubit = LQubit(lattice, l[0], l[1], True)
cycle = SurfaceCodeCycle(lattice)
num_qubits = WIDTH * HEIGHT

In [14]:
qc = ConstrainedQuantumCircuit(lattice, WIDTH * HEIGHT)
qc.add_register(ClassicalRegister(num_qubits))
qc = qc.compose(cycle._circuit(1), list(range(num_qubits)), list(range(num_qubits)))

qc.add_register(ClassicalRegister(num_qubits + len(qubit.route(l[0],l[1])) // 2))
qc = qc.compose(qubit.alt_initialize(cycle), list(range(num_qubits)), list(range(num_qubits, 2 * num_qubits + len(qubit.route(l[0],l[1])) // 2)))



qc.add_register(ClassicalRegister(num_qubits))
qc = qc.compose(cycle._circuit(1), list(range(num_qubits)), list(range(2 * num_qubits + len(qubit.route(l[0],l[1])) // 2, 3 * num_qubits + len(qubit.route(l[0],l[1])) // 2)))

qc.add_register(ClassicalRegister(2))
qc = qc.compose(qubit.alt_measure(), list(range(num_qubits)), list(range(3* num_qubits + len(qubit.route(l[0],l[1])) // 2, 3 * num_qubits + len(qubit.route(l[0],l[1])) // 2 + 2)))

In [15]:
provider = IBMProvider(token='e521151100e3c7cafa35bc9467cced1b3ed68c5279e3b75e913f731410b55a444bc7f7053bedd0d97930cbec58783746cda23b32655944c5933140067480595c')
backend = provider.get_backend('ibm_brisbane')
#backend = least_busy(provider.backends(simulator=False, operational=True))
backend

<IBMBackend('ibm_brisbane')>

In [16]:
built = transpile(qc, backend)

In [17]:
job = backend.run(built, shots=100)
job.job_id()

'cpvbbd51tcz0008mjcg0'

# Bit flip

In [7]:
WIDTH, HEIGHT = 3, 9
l = (7, 19)

lattice = SquareLattice(WIDTH, HEIGHT)
qubit = LQubit(lattice, l[0], l[1], True)
cycle = SurfaceCodeCycle(lattice)
num_qubits = WIDTH * HEIGHT

In [8]:
qc = ConstrainedQuantumCircuit(lattice, WIDTH * HEIGHT)
qc.add_register(ClassicalRegister(num_qubits))
qc = qc.compose(cycle._circuit(1), list(range(num_qubits)), list(range(num_qubits)))

qc.add_register(ClassicalRegister(num_qubits + len(qubit.route(l[0],l[1])) // 2))
qc = qc.compose(qubit.alt_initialize(cycle), list(range(num_qubits)), list(range(num_qubits, 2 * num_qubits + len(qubit.route(l[0],l[1])) // 2)))

qc.add_register(ClassicalRegister(num_qubits))
qc = qc.compose(cycle._circuit(1), list(range(num_qubits)), list(range(2 * num_qubits + len(qubit.route(l[0],l[1])) // 2, 3 * num_qubits + len(qubit.route(l[0],l[1])) // 2)))

qc = qc.compose(qubit.circle_gate())

qc.add_register(ClassicalRegister(num_qubits))
qc = qc.compose(cycle._circuit(1), list(range(num_qubits)), list(range(3 * num_qubits + len(qubit.route(l[0],l[1])) // 2, 4 * num_qubits + len(qubit.route(l[0],l[1])) // 2)))

qc.add_register(ClassicalRegister(2))
qc = qc.compose(qubit.alt_measure(), list(range(num_qubits)), list(range(4 * num_qubits + len(qubit.route(l[0],l[1])) // 2, 4 * num_qubits + len(qubit.route(l[0],l[1])) // 2 + 2)))

In [9]:
backend = least_busy(provider.backends(simulator=False, operational=True))
backend

<IBMBackend('ibm_kyoto')>

In [10]:
built = transpile(qc, backend)

In [11]:
job = backend.run(built, shots=100)
job.job_id()

IBMBackendApiError: 'Error submitting job: \'409 Client Error: Conflict for url: https://api.quantum.ibm.com/runtime/jobs. {"errors":[{"message":"You have reached the limit of 3 pending  jobs. Please wait for a job to complete or cancel one before submitting anything new.","code":3458,"solution":"Wait until some previous jobs were finished. You can cancel pending jobs to run new jobs.","more_info":"https://docs.quantum-computing.ibm.com/errors"}]}\''